In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/pytorch_colab/rolling_and_plot.py .
!cp /content/drive/MyDrive/pytorch_colab/220617_soc.csv .

In [ ]:
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
# # !pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor, Lambda

# import torch_xla
# import torch_xla.core.xla_model as xm

from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from dataclasses import dataclass

In [ ]:
import numpy as np
import pandas as pd
from rolling_and_plot import *

%reload_ext autoreload
%autoreload 2

In [ ]:
# Get cpu or gpu device for training.
# device = xm.xla_device()
device = torch.device("cuda")
print(f"Using {device} device")

# TOC

* [Preprocessing](#pre)

* [Data Loading](#dload)

* [Models](#model)

<a id="pre"></a>
# PreProcessing 

## <a id="g">G class</a>

In [ ]:
@dataclass
class G:
    num_features = 3 # current, voltage, and soc at t-1
    lstm_nodes = 256
    window_size = 256
    batch_size = 32
    epochs = 5

In [ ]:
# from google.colab import files
file = pd.read_csv("/content/220617_soc.csv")

In [ ]:
data_plot(data = [file],
          title="OCV v SOC",
          x = ["test time (sec)"],
          y = ["soc"],
          markers = "lines",
          color = "darkorchid",
          x_title = "Test Time (sec)",
          y_title = "SOC"
         )

In [ ]:
file = normalize(file.loc[:,["current","voltage","soc"]].iloc[::5])
#uses sklearn.preprocessing

In [ ]:
x_train, x_test, y_train, y_test = rolling_split(file, G.window_size)
x_train.shape, x_test.shape, y_train.shape, y_test.shape
#uses sklearn.model_selection

In [ ]:
# np.array_split(train,len(train)//6) not applicable here

# Data Loader <a id="dload"></a>

In [ ]:
class BatterySet(Dataset):
    def __init__(self, x: np.ndarray, y: np.ndarray):
    
        self.logits = torch.from_numpy(x.squeeze()).to(device)
        self.labels = torch.from_numpy(y).to(device)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (self.logits[idx], self.labels[idx])

In [ ]:
train_dataloader = BatterySet(x_train, y_train)
test_dataloader = BatterySet(x_test, y_test)

In [ ]:
train_dataloader = DataLoader(train_dataloader, batch_size=G.batch_size, shuffle=True, drop_last = True)
test_dataloader = DataLoader(test_dataloader, batch_size=G.batch_size, shuffle=False, drop_last = True)

In [ ]:
for X,y in train_dataloader:
    print(f"Shape of X [window, features]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
for batch, (x,y) in enumerate(test_dataloader.dataset):
    print(batch,x,y)
    break

# Creating Models <a id="model"></a>

Go to [G class](#g)

According to Song et al. in the article [Combined CNN-LSTM Network for State-of-Charge Estimation of Lithium-Ion Batteries](https://ieeexplore.ieee.org/abstract/document/8754752) a single-layer LSTM is enough to map non-linearity of SOC.

In [ ]:
# # Get cpu or gpu device for training.
# device = torch.device("mps")
# print(f"Using {device} device")

# Define model
class LSTMNetwork(nn.Module):
    
    def __init__(self):
        super(LSTMNetwork, self).__init__()

        self.batch_norm1 = nn.BatchNorm1d(G.batch_size, momentum = 0.15, eps=1.0e-5)

        self.cnn = nn.Conv1d(G.window_size, G.window_size, 3,
                             stride = 1, padding = "same")
        for name, param in self.cnn.named_parameters():
            if 'bias' in name:
                nn.init.uniform_(param, a=0.01, b=0.09)
            elif 'weight' in name:
                nn.init.kaiming_normal_(param, nonlinearity = "relu") # equivalent to He Normalization

        self.lstm = nn.LSTM(G.num_features, G.lstm_nodes, 1, batch_first = True)
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.uniform_(param, a=0.001, b=0.04)
            elif 'weight_ih' in name:
                nn.init.kaiming_normal_(param, nonlinearity = "relu")
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)

        self.linear_stack = nn.Sequential(
            nn.Linear(G.lstm_nodes, G.lstm_nodes), # shape == (G.batch_size, G.lstm_nodes)
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(G.lstm_nodes, G.lstm_nodes), # shape == (G.batch_size, G.lstm_nodes)
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(G.lstm_nodes, G.lstm_nodes), # shape == (G.batch_size, G.lstm_nodes)
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(G.lstm_nodes, G.lstm_nodes), # shape == (G.batch_size, G.lstm_nodes)
            nn.ReLU(),
            nn.Linear(G.lstm_nodes, 1), # shape == (G.batch_size, 1)
            nn.ReLU()
        )
        for i in range(0, len(self.linear_stack), 2):
            for name, param in self.linear_stack[i].named_parameters():
                if 'bias' in name:
                    nn.init.uniform_(param,
                                     a=(0.1 / (i/2 + 1)),
                                     b=(0.9/ (i/2 + 1))
                                    )
                elif 'weight' in name:
                    nn.init.kaiming_normal_(param, nonlinearity = "relu")

#     def l2_normalize(self, x, dim = 1):
#         "apparently weight decay in the optimize functions does l2 regularization so this is unnecessary"
#         return nn.functional.normalize(x, p = 2.0 , dim = dim)
    
    def forward(self, x): # assert(x.shape == (G.batch_size, G.window_size, G.num_features))
        #cnn
        x_out = self.cnn(x) # assert(x_out.shape == (G.batch_size, G.window_size, G.num_features))
        
        #lstm
        x_out, (h_n_lstm, c_n)  = self.lstm(x_out) # assert(h_n_lstm.shape == (1, G.batch_size, G.lstm_nodes))
        h_n_lstm = self.batch_norm1(h_n_lstm) # assert(h_n_lstm.shape == (1, G.batch_size, G.lstm_nodes))
        
        #Dense Layers
        # send the final lstm layer's hidden state values to the Dense Layers
        out = self.linear_stack(h_n_lstm.squeeze())

        return out # (G.batch_size, 1)

Can load a pretrained model, the cell is after the optimizer cell.

You need to run the cell right below this first though.

In [ ]:
model = LSTMNetwork().to(device)
print(model)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    perc_error = 0
    for batch, (x,y) in enumerate(dataloader):
        optimizer.zero_grad() #resets the gradient graph, a pytorch shortcoming that is required
        
        #forward
        predict = model(x)
        loss = loss_fn(predict, y).mean(0) # assert(loss.shape == (1))

        #backward
        loss.backward() #Backprop on 1-D Dense Layer output doesnt work on M1 Mac's "mps" device
                        # This is apparently fixed in PyTorch 1.13.0 which has not been officially released yet
                        # I am also too lazy to build 1.13.0 from the master repo on their github
        optimizer.step()
        ##### For OneCycleLR:
        scheduler.step()

        if loss.isnan():
            print("loss was NaN")
            break

        if batch % (size // 3) == 0:
            loss, current = loss.item(), batch
            print(f"batch mean loss: {loss:>7f}  [{current:4d}/{size:4d}]")

        with torch.no_grad(): #used to check bias and variance by comparing with test set
            perc_error += torch.mean(torch.abs(predict - y) / (y+ 1e-2) * 100, 0).mean(0)

    ###### For ReduceLROnPlateau:
    # scheduler.step(loss) # reduces the learning rate based on a scheduler

    perc_error /= size
    print(f"Train Error: \nAverage Accuracy: {100 - perc_error}%")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader)
    num_batches = size // G.batch_size
    test_loss, perc_error = 0.0, 0.0
    counter = 0
    with torch.no_grad(): #doesnt update parameters (we are testing not training)
        for x,y in dataloader:
            predict = model(x).reshape(y.shape)
            test_loss += loss_fn(predict, y).mean(0).item()
            perc_error += torch.mean(torch.abs(predict - y) / (y+ 1e-2) * 100, 0).mean(0)
           
            counter += 1
            if counter % (size // 2) == 0:
                print(f"{counter} / {size} tested")
            
            
    test_loss /= size
    perc_error /= size
    print(f"Test Error: \nAverage Accuracy: {100 - perc_error}%, Avg Loss: {test_loss:>8f}\n")
    return perc_error, test_loss

## Log Cosh Loss Function

In [ ]:
def log_cosh_loss(y_pred: torch.tensor, y_ground: torch.tensor) -> torch.tensor:
    x = y_pred - y_ground
    return x + torch.nn.functional.softplus(-2. * x) - np.log(2.0)

class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,
                y_pred: torch.Tensor,
                y_true: torch.Tensor) -> torch.Tensor:
        return log_cosh_loss(y_pred, y_true)

**Literature**

*Loss*<br>
The LogCoshLoss is the Loss function used by Hannan et al. in their article in the Journal *Nature*: [Deep learning approach towards accurate state of charge estimation for lithium-ion batteries using self-supervised transformer model](https://www.nature.com/articles/s41598-021-98915-8).

However they used a Transformer Network which I am doing as well in another project implemented in Tensorflow.

*Learning Rate*<br>
The OneCycle learning rate scheduler with cosine annealing introduced by Leslie N. Smith in his paper [A disciplined approach to neural network hyper-parameters: Part 1 -- learning rate, batch size, momentum, and weight decay](https://doi.org/10.48550/arXiv.1803.09820), seems to be the best scheduler according to Fast.AI

The original 3-phase apporach seems to work significantly better than the 2-phase method by Fast.AI

In [ ]:
# loss_fn = nn.SmoothL1Loss()
# loss_fn = nn.HuberLoss()
# # loss_fn = nn.MSELoss()
# loss_fn = LogCoshLoss()
# loss_fn = nn.KLDivLoss()

#I am trying no weight decay for a bit
# optimizer = torch.optim.Adam(model.parameters(),
#                              lr = 0.01,
#                              weight_decay= 1e-4
#                             )

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min",
#                                                        factor = 0.6,
#                                                        patience = 4, cooldown = 1,
#                                                        min_lr = 7e-11,
#                                                        verbose=True)

#OneCycle scheduler needs step() to be called after every batch
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                0.0008, #max_lr
                                                epochs = G.epochs,
                                                steps_per_epoch = len(train_dataloader),
                                                anneal_strategy = "cos", #cosine annealing
                                                div_factor = 35,
                                                three_phase = True,
                                                verbose = False
                                                )

In [ ]:
model.load_state_dict(torch.load("drive/MyDrive/pytorch_colab/cnn_model_state_dict.pth", map_location = device))
# optimizer.load_state_dict(torch.load("drive/MyDrive/pytorch_colab/cnn_optimizer_state_dict.pth", map_location = device))
model.train()
# model.eval()

# Training

In [ ]:
for epoch in range(1, G.epochs + 1):
    print(f"Epoch {epoch}/{G.epochs}\n----------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

    if (epoch != 0) and (epoch % 50 == 0):
        torch.save(model.state_dict(), "drive/MyDrive/pytorch_colab/two_phase_model_state_dict.pth")
        torch.save(optimizer.state_dict(), "drive/MyDrive/pytorch_colab/two_phase_optimizer_state_dict.pth")
        print("Saved the model parameters\n")

print("Completed")

In [ ]:
torch.save(model.state_dict(), "drive/MyDrive/pytorch_colab/three_phase_model_state_dict.pth")
torch.save(optimizer.state_dict(), "drive/MyDrive/pytorch_colab/three_phase_optimizer_state_dict.pth")

In [ ]:
pred = []
with torch.no_grad():
    for x,y in test_dataloader:
        pred.append(model(x))

aggregate = []
for i in pred: #this way is faster than list comprehension below
    aggregate.extend(i)
print(max(aggregate), min(aggregate))


# aggregate = [unit for batch in pred for unit in batch]
# print(max(aggregate), min(aggregate))


In [ ]:
np_aggregate = np.array([p.detach().cpu().numpy() for p in aggregate])
np_labels = torch.clone(test_dataloader.dataset.labels).detach().cpu().numpy()[:len(np_aggregate)]

visualize = pd.DataFrame(data={"pred": np_aggregate.squeeze(),
                               "labels": np_labels.squeeze()})

visualize.sort_values("labels", inplace=True)
visualize.reset_index(drop=True)

visualize["point"] = list(range(1, len(visualize)+1))

In [ ]:
visualize

In [ ]:
data_plot(data = visualize,
          x = [["point","point"]],
          y = [["pred","labels"]],
          x_title = "Data Point",
          y_title = "SOC",
          title = "Predicted vs Actual SOC",
          name = [["predictions","labels"]],
          mode = [["markers","markers"]],
          color = [["red","yellow"]]
         )